In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    setting = yaml.load(yaml_file)

path = make_path_dict(setting)

In [ ]:
feature_data = setting["feature_data"]

if feature_data is None:

    feature_data = {}

else:

    for data_name, data_dict in feature_data.items():

        feature_data[data_name]["df"] = pd.read_table(
            data_dict["file_path"], index_col=0
        )

    print(feature_data.keys())

In [ ]:
sample_data = setting["sample_data"]

if sample_data is None:

    sample_data = {}

else:

    for data_name, data_dict in sample_data.items():

        sample_data[data_name]["df"] = pd.read_table(
            data_dict["file_path"], index_col=0
        )

    print(sample_data.keys())

In [ ]:
make_match_panels_kwargs = dict(
    n_job=setting["max_n_job"],
    n_required_for_match_function=setting["n_required_for_match_function"],
    extreme_feature_threshold=setting["extreme_feature_threshold"],
    n_sampling=setting["n_sampling"],
    n_permutation=setting["n_permutation"],
    plot_std=setting["plot_std"],
)

In [ ]:
for w_or_h, targets, data_dicts in (
    ("w", pd.read_table(path["w.tsv"], index_col=0).T, feature_data),
    ("h", pd.read_table(path["h.tsv"], index_col=0), sample_data),
):
    directory_path = path["{}|match/".format(w_or_h)]

    plotly_directory_path = path["plotly|{}_match/".format(w_or_h)]

    ccal.make_match_panels(
        targets,
        data_dicts,
        directory_path=directory_path,
        plotly_directory_path=plotly_directory_path,
        **make_match_panels_kwargs,
    )

In [ ]:
for w_or_h, data_dicts in (("w", feature_data), ("h", sample_data)):

    cluster_x_column_file_path = path["{}|cluster_x_column.tsv".format(w_or_h)]

    targets = pd.read_table(cluster_x_column_file_path, index_col=0)

    directory_path = path["{}|hcc|match/".format(w_or_h)]

    plotly_directory_path = path["plotly|{}_match/".format(w_or_h)]

    ccal.make_match_panels(
        targets,
        data_dicts,
        target_type="binary",
        cluster_within_category=targets.shape[1] < setting["plot_cluster_max_size"],
        directory_path=directory_path,
        plotly_directory_path=plotly_directory_path,
        **make_match_panels_kwargs,
    )